# Hyperparameter Optimization
Hyperparameters dictate the parameters of the training process and the architecture of the model itself. For example, the 
number of random trees is a hyperparameter for a **random forest**. In contrast, a learned parameter for a **random forest** is the set of features that is contained in a single node (in a single tree) and the cutoff values for each of those features that determines how the data is split at that node. A full discussion of hyperparameter optimization can be found on **[Wikipedia](https://en.wikipedia.org/wiki/Hyperparameter_optimization)**.

The choice of hyperparameters strongly influences model performance,
so it is important to be able to optimize them as well. **[AMPL](https://github.com/ATOMScience-org/AMPL)**
offers a variety of hyperparameter optimization methods including
random sampling, grid search, and Bayesian optimization. Please refer to the parameter documentation 
**[page](https://github.com/ATOMScience-org/AMPL#hyperparameter-optimization)** for further information.

In this tutorial we demonstrate the following:
- Build a parameter dictionary to perform a hyperparameter search for a **random forest** using Bayesian optimization.
- Perform the optimization process.
- Review the results

We will use these **[AMPL](https://github.com/ATOMScience-org/AMPL)** functions:
- [parse_params](https://ampl.readthedocs.io/en/latest/utils.html#utils.hyperparam_search_wrapper.parse_params)
- [build_search](https://ampl.readthedocs.io/en/latest/utils.html#utils.hyperparam_search_wrapper.build_search)
- [run_search](https://ampl.readthedocs.io/en/latest/utils.html#utils.hyperparam_search_wrapper.HyperOptSearch.run_search)
- [get_filesystem_perf_results](https://ampl.readthedocs.io/en/latest/pipeline.html#pipeline.compare_models.get_filesystem_perf_results)

The first three functions in the above list come from the `hyperparameter_search_wrapper` module. 

## Set Up Directories

Here we set up a few important variables corresponding to required directories and specific features for the **hyperparameter optimization (HPO)** process. Then, we ensure that the directories are created before saving models into them.

|Variable|Description|
|---|---|
|`dataset_key`|The relative path to the dataset you want to use for HPO|
|`descriptor_type`|The type of features you want to use during HPO|
|`model_dir`|The directory where you want to save all of the models|
|`best_model_dir`|For Bayesian optimization, the winning model is saved in this separate folder|
|`split_uuid`|The presaved split uuid from **Tutorial 2, "Splitting Datasets for Validation and Testing"**|

In [13]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

import os

dataset_key='dataset/final_data_curated.csv'
descriptor_type = 'rdkit_raw'
model_dir = 'dataset/final_models'
best_model_dir = 'dataset/final_models/best_models'
split_uuid = "f346a459-b654-4e1a-9bb4-8c4207a19b1f"


if not os.path.exists(f'./{best_model_dir}'):
    os.mkdir(f'./{best_model_dir}')
    
if not os.path.exists(f'./{model_dir}'):
    os.mkdir(f'./{model_dir}')

To run a hyperparameter search, we first create a parameter dictionary with parameter settings that will be common to all models, along with some special parameters that control the search and indicate which parameters will be varied and how. The table below describes the special parameter settings for our random forest search.

## Parameter Dictionary Settings

|Parameter|Description|
|---|---|
|`'hyperparam':'True'`|This setting indicates that we are performing a hyperparameter search instead of just training one model.|
|`'previously_featurized':'True'`|This tells **[AMPL](https://github.com/ATOMScience-org/AMPL)** to search for previously generated features in `../dataset/scaled_descriptors` instead of regenerating them on the fly.|
|`'search_type':'hyperopt'`|This specifies the hyperparameter search method. Other options include `grid`, `random`, and `geometric`. Specifications for each hyperparameter search method is different, please refer to the full documentation. Here we are using the Bayesian optimization method.|
|`'model_type':'RF\|10'`|This means **[AMPL](https://github.com/ATOMScience-org/AMPL)** will try 10 times to find the best set of hyperparameters using **random forests**. In practice, this parameter could be set to 100 or more.|
|`'rfe':'uniformint\|8,512'`|The Bayesian optimizer will uniformly search between 8 and 512 for the best number of random forest estimators. Similarly `rfd` stands for **random forest depth** and `rff` stands for **random forest features**.|
|`result_dir`|Now expects two parameters. The first directory will contain the best trained models while the second directory will contain all models trained in the search.|

Regression models are optimized to maximize the $R^2$ and
classification models are optimized using area under the 
receiver operating characteristic curve.
A full list of parameters can be found on our
**[github](https://github.com/ATOMScience-org/AMPL/blob/master/atomsci/ddm/docs/PARAMETERS.md)**.

In [14]:
params = {
    "hyperparam": "True",
    "prediction_type": "regression",

    "dataset_key": dataset_key,
    "id_col": "compound_id",
    "smiles_col": "base_rdkit_smiles",
    "response_cols": "Y",

    "splitter":"scaffold",
    "split_uuid": split_uuid,
    "previously_split": "True",

    "featurizer": "computed_descriptors",
    "descriptor_type" : descriptor_type,
    "transformers": "True",

    "search_type": "hyperopt",
    "model_type": "RF|10",
    "rfe": "uniformint|8,512",
    "rfd": "uniformint|6,32",
    "rff": "uniformint|8,200",

    "result_dir": f"./{best_model_dir},./{model_dir}"
}

## Run Hyperparameter Search
In **Tutorial 3, "Train a Simple Regression Model"**, we directly imported the `parameter_parser` and `model_pipeline` objects to parse the `config` dict and train a single model. Here, we use `hyperparameter_search_wrapper` to handle many models for us. First we build the search by creating a list of parameters to use, and then we run the search.

In [19]:
import atomsci.ddm.utils.hyperparam_search_wrapper as hsw
import importlib
importlib.reload(hsw)
ampl_param = hsw.parse_params(params)
hs = hsw.build_search(ampl_param)
hs.run_search()

model_performance|train_r2|train_rms|valid_r2|valid_rms|test_r2|test_rms|model_params|model

xgb_gamma: 0.16049048354303894, xgb_learning_rate: 0.7937793192040994, xgb_max_depth: 6, xgb_colsample_bytree: 1.0, xgb_subsample: 1.0, xgb_n_estimators: 100, xgb_min_child_weight: 1.0
xgboost model with computed_descriptors and rdkit_raw                                                    
  0%|                                                             | 0/10 [00:00<?, ?trial/s, best loss=?]

2024-07-19 14:44:25,438 Previous dataset split restored
/opt/atomsci-env/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [14:44:25] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)

/opt/atomsci-env/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [14:44:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_gpus", "silent" } are not used.

  warnings.warn(smsg, UserWarning)



model_performance|0.917|0.271|0.091|1.067|0.052|1.147|0.16049048354303894_0.7937793192040994_6_1.0_1.0_100_1.0|./dataset/final_models/final_data_curated_model_35f616fd-0468-4b0e-87a2-5b55960da2d4.tar.gz

xgb_gamma: 0.1603923077618593, xgb_learning_rate: 5.428415252269048, xgb_max_depth: 6, xgb_colsample_bytree: 1.0, xgb_subsample: 1.0, xgb_n_estimators: 100, xgb_min_child_weight: 1.0
xgboost model with computed_descriptors and rdkit_raw                                                    
 10%|███▌                               | 1/10 [00:00<00:05,  1.75trial/s, best loss: 0.9092469922305699]

2024-07-19 14:44:26,010 Previous dataset split restored
/opt/atomsci-env/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [14:44:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_gpus", "silent" } are not used.

  warnings.warn(smsg, UserWarning)



model_performance|0.000|100.000|0.000|100.000|0.000|100.000|0.1603923077618593_5.428415252269048_6_1.0_1.0_100_1.0|./dataset/final_models/final_data_curated_model_27514994-b5aa-48b7-9b28-546e255b3cf7.tar.gz

xgb_gamma: 0.09115948049300145, xgb_learning_rate: 0.15604420029845428, xgb_max_depth: 6, xgb_colsample_bytree: 1.0, xgb_subsample: 1.0, xgb_n_estimators: 100, xgb_min_child_weight: 1.0
xgboost model with computed_descriptors and rdkit_raw                                                    
 20%|███████                            | 2/10 [00:01<00:04,  1.93trial/s, best loss: 0.9092469922305699]

...

model_performance|0.951|0.209|0.129|1.044|0.099|1.118|0.05848213310584516_0.793144063576144_6_1.0_1.0_100_1.0|./dataset/final_models/final_data_curated_model_6b8e3e51-a7ac-4006-be7f-8d1a80cb04e7.tar.gz

xgb_gamma: 0.15683305856616978, xgb_learning_rate: 0.8853426792430782, xgb_max_depth: 6, xgb_colsample_bytree: 1.0, xgb_subsample: 1.0, xgb_n_estimators: 100, xgb_min_child_weight: 1.0
xgboost model with computed_descriptors and rdkit_raw                                                    
 80%|████████████████████████████       | 8/10 [00:04<00:01,  1.73trial/s, best loss: 0.6533530852831401]

2024-07-19 14:44:29,991 Previous dataset split restored
/opt/atomsci-env/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [14:44:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_gpus", "silent" } are not used.

  warnings.warn(smsg, UserWarning)



model_performance|0.925|0.257|0.064|1.083|-0.020|1.190|0.15683305856616978_0.8853426792430782_6_1.0_1.0_100_1.0|./dataset/final_models/final_data_curated_model_1c597f0b-b8f7-40e1-9ba7-c6041d54108a.tar.gz

xgb_gamma: 0.04009766682360752, xgb_learning_rate: 0.387820507539886, xgb_max_depth: 6, xgb_colsample_bytree: 1.0, xgb_subsample: 1.0, xgb_n_estimators: 100, xgb_min_child_weight: 1.0
xgboost model with computed_descriptors and rdkit_raw                                                    
 90%|███████████████████████████████▌   | 9/10 [00:05<00:00,  1.76trial/s, best loss: 0.6533530852831401]

2024-07-19 14:44:30,535 Previous dataset split restored
/opt/atomsci-env/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [14:44:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_gpus", "silent" } are not used.

  warnings.warn(smsg, UserWarning)



model_performance|0.925|0.258|0.315|0.926|0.306|0.981|0.04009766682360752_0.387820507539886_6_1.0_1.0_100_1.0|./dataset/final_models/final_data_curated_model_f4473d47-d9c6-456b-99ab-10cf319b5b98.tar.gz

100%|██████████████████████████████████| 10/10 [00:05<00:00,  1.76trial/s, best loss: 0.6533530852831401]
Generating the performance -- iteration table and Copy the best model tarball.
Best model: ./dataset/final_models/final_data_curated_model_b6763e7f-d080-4cb1-863e-f9f177fc7433.tar.gz, valid R2: 0.34664691471685993


The top scoring model will be saved in `dataset/SLC6A3_models/best_models` along with a csv file
containing regression performance for all trained models.

All of the models are saved in `dataset/SLC6A3_models`. These models can be
explored using `get_filesystem_perf_results`. A full analysis of the hyperparameter performance is explored in **Tutorial 6, "Compare models to select the best hyperparameters"**.

In [20]:
import atomsci.ddm.pipeline.compare_models as cm

result_df = cm.get_filesystem_perf_results(
    result_dir=model_dir,
    pred_type='regression'
)

# sort by validation r2 score to see top performing models
result_df = result_df.sort_values(by='best_valid_r2_score', ascending=False)
result_df[['model_uuid','model_parameters_dict','best_valid_r2_score','best_test_r2_score']].head()

Found data for 29 models under dataset/final_models


,model_uuid,model_parameters_dict,best_valid_r2_score,best_test_r2_score
19,a0ac4513-a33a-4a39-a9f7-113d9b852864,"{""best_epoch"": 12, ""dropouts"": [0.342082063392...",0.403875,0.400500
24,906072ff-83c1-4549-bf98-d473387e897e,"{""best_epoch"": 57, ""dropouts"": [0.016191935195...",0.398768,0.383271
26,230e1138-fcac-4792-80aa-682657277c9f,"{""best_epoch"": 73, ""dropouts"": [0.113310441604...",0.398239,0.404253
21,750b683a-07ce-4cd1-b0a7-c2c423f26e4c,"{""best_epoch"": 19, ""dropouts"": [0.033750916470...",0.378872,0.357292
15,b6763e7f-d080-4cb1-863e-f9f177fc7433,"{""xgb_colsample_bytree"": 1.0, ""xgb_gamma"": 0.0...",0.346647,0.289184


In [21]:
result_df.to_csv('./final_hypers.csv', index=False)

## Examples of Other Parameter Sets
Below are some parameters that can be used for **neural networks**, 
**[XGBoost](https://en.wikipedia.org/wiki/XGBoost)** models, 
**fingerprint splits** and **[ECFP](https://pubs.acs.org/doi/10.1021/ci100050t)** features.
Each set of parameters can be used to replace the parameters above. 
Trying them out is left as an exercise for the reader.

#### Neural Network Hyperopt Search

|Parameter|Description|
|---|---|
|`lr`| This controls the learning rate. loguniform\|-13.8,-3 means the logarithm of the learning rate is uniformly distributed between -13.8 and -3.|
|`ls` |This controls layer sizes. 3\|8,512 means 3 layers with sizes ranging between 8 and 512 neurons. A good strategy is to start with a fewer layers and slowly increase the number until performance plateaus.| 
|`dp`| This controls dropout. 3\|0,0.4 means 3 dropout layers with probability of zeroing a weight between 0 and 40%. This needs to match the number of layers specified with `ls` and should range between 0% and 50%. |
|`max_epochs`| This controls how long to train each model. Training for more epochs increases runtime, but allows models more time to optimize. |

In [16]:
params = {
    "hyperparam": "True",
    "prediction_type": "regression",

    "dataset_key": dataset_key,
    "id_col": "compound_id",
    "smiles_col": "base_rdkit_smiles",
    "response_cols": "Y",

    "splitter":"scaffold",
    "split_uuid": split_uuid,
    "previously_split": "True",

    "featurizer": "computed_descriptors",
    "descriptor_type" : descriptor_type,
    "transformers": "True",

    ### Use a NN model
    "search_type": "hyperopt",
    "model_type": "NN|10",
    "lr": "loguniform|-13.8,-3",
    "ls": "uniformint|3|8,512",
    "dp": "uniform|3|0,0.4",
    "max_epochs":100,
    ###

    "result_dir": f"./{best_model_dir},./{model_dir}"
}

#### XGBoost
- `xgbg` Stands for `xgb_gamma` and controls the minimum loss 
reduction required to make a further partition on a leaf node of the tree.
- `xgbl` Stands for `xgb_learning_rate` and controls the boosting 
learning rate searching domain of  **[XGBoost](https://en.wikipedia.org/wiki/XGBoost)** models.

In [18]:
params = {
    "hyperparam": "True",
    "prediction_type": "regression",

    "dataset_key": dataset_key,
    "id_col": "compound_id",
    "smiles_col": "base_rdkit_smiles",
    "response_cols": "Y",

    "splitter":"scaffold",
    "split_uuid": split_uuid,
    "previously_split": "True",

    "featurizer": "computed_descriptors",
    "descriptor_type" : descriptor_type,
    "transformers": "True",

    ### Use an XGBoost model
    "search_type": "hyperopt",
    "model_type": "xgboost|10",
    "xgbg": "uniform|0,0.2",
    "xgbl": "loguniform|-2,2",
    ###

    "result_dir": f"./{best_model_dir},./{model_dir}"
}

#### Fingerprint Split
This trains an  **[XGBoost](https://en.wikipedia.org/wiki/XGBoost)** model using a
**fingerprint split**. The fingerprint split is provided with the dataset files. 

In [ ]:
fp_split_uuid="be60c264-6ac0-4841-a6b6-41bf846e4ae4"

params = {
    "hyperparam": "True",
    "prediction_type": "regression",

    "dataset_key": dataset_key,
    "id_col": "compound_id",
    "smiles_col": "base_rdkit_smiles",
    "response_cols": "avg_pKi",

    ### Use a fingerprint split
    "splitter":"fingerprint",
    "split_uuid": fp_split_uuid,
    "previously_split": "True",
    ###

    "featurizer": "computed_descriptors",
    "descriptor_type" : descriptor_type,
    "transformers": "True",

    "search_type": "hyperopt",
    "model_type": "xgboost|10",
    "xgbg": "uniform|0,0.2",
    "xgbl": "loguniform|-2,2",

    "result_dir": f"./{best_model_dir},./{model_dir}"
}

#### ECFP Features
This uses an  **[XGBoost](https://en.wikipedia.org/wiki/XGBoost)** model with **[ECFP fingerprints](https://pubs.acs.org/doi/10.1021/ci100050t)** features and a **scaffold split**.

In [ ]:
params = {
    "hyperparam": "True",
    "prediction_type": "regression",

    "dataset_key": dataset_key,
    "id_col": "compound_id",
    "smiles_col": "base_rdkit_smiles",
    "response_cols": "avg_pKi",

    "splitter":"scaffold",
    "split_uuid": split_uuid,
    "previously_split": "True",

    ### Use ECFP Features
    "featurizer": "ecfp",
    "ecfp_radius" : 2,
    "ecfp_size" : 1024,
    "transformers": "True",
    ###

    "search_type": "hyperopt",
    "model_type": "xgboost|10",
    "xgbg": "uniform|0,0.2",
    "xgbl": "loguniform|-2,2",

    "result_dir": f"./{best_model_dir},./{model_dir}"
}

In **Tutorial 6, "Compare Models to Select the Best Hyperparameters"**, we analyze the performance of these large sets of models to select the best hyperparameters for production models.

If you have specific feedback about a tutorial, please complete the **[AMPL Tutorial Evaluation](https://forms.gle/pa9sHj4MHbS5zG7A6)**.